In [1]:
from doc_parse import chunk, read_and_process_excel, logger

In [2]:
import pandas as pd
from langchain_chroma import Chroma
import chromadb

In [3]:
pdf_files = ['./data/zhidu_employee.pdf', './data/zhidu_travel.pdf']
excel_files = ['./data/zhidu_detail.xlsx']

In [4]:
doc_data = []
for pdf_file_name in pdf_files:
    res = chunk(pdf_file_name)
    doc_data.extend(res)

print(f"解析文档个数：{len(doc_data)}")
for doc in doc_data:
    print("#"*20)
    print(doc.page_content)

解析文档个数：33
####################
教职工考勤管理制度
第一节 适用范围
1、本制度包括了考勤、休假、加班等方面的规定。
2、本制度适用于学校全体教职员工。
第二节 考勤规定
1、学校的工作时间由学校决定并公布。学校内除特殊岗位特别规定外，全体教职员工
均应严格执行学校的作息时间表，不迟到、不早退、不中途离校。
工作时间：星期一至星期四 7:55-16:55 星期五 7:55-16:15
####################
工作时间：星期一至星期四 7:55-16:55 星期五 7:55-16:15
2、所有教职工实行考勤打卡制度，工作日内，每天需打卡两次，上午上班一次和下午
下班一次。
3、教职员工因故(特殊情况除外)晚到或早退，应事先履行请假手续，经批准后方可离
校。原则上，员工请假无论时间长短、假期形式，除急诊病假或突发事件外，一律需按照请
假流程。
请假，需事先在钉钉系统中提交申请。
有效的请假流程为：
####################
假流程。
请假，需事先在钉钉系统中提交申请。
有效的请假流程为：
（1）员工休假必须事先向部门负责人申请，将工作交接清楚方可休假；
（2）2天以内的假期必须经过部门分管领导审批；
（3）3天以上的假期必须经过校长审批。
如遇紧急情况，口头申请请假的，应在上班后两天内办理补请假手续，未在规定时间内
办理的，逾期无效，按旷工处理。
####################
办理的，逾期无效，按旷工处理。
4、无工作理由，超过上班时间到岗的，视为迟到；未到下班时间提前离校的，视为早
退；中途未经批准离校，视为旷工；迟到、早退、旷工者按照相关办法处理。
5、考勤员徇私舞弊、弄虚作假的，按学校奖惩规定处理。情节严重的，予以通报批评
直至解聘。
第三节 休假规定
休假分以下八种：事假、病假、婚假、丧假、产假、哺乳假、工伤假、调休。
####################
直至解聘。
第三节 休假规定
休假分以下八种：事假、病假、婚假、丧假、产假、哺乳假、工伤假、调休。
1、事假：（1）请假必须严格按照学校规定的请假程序，应由本人以书面形式，应在前
两日办理手续，特殊情况应通过电话或者口头请假，应在事假结束于2个工作日内完成补办
请假手续。未经请假或请假未准而擅自离岗者，以旷工论处。（2）事假最小

In [5]:
from langchain_core.documents import Document

for excel_file_name in excel_files:
    data = read_and_process_excel(excel_file_name)
    df = pd.DataFrame(data[8:], columns=data[7])
    data_excel = df.drop(columns=df.columns[11:17])
    doc_excel = Document(
        page_content = data_excel.to_markdown(index=False).replace(" ", ""),
        metadata = {"source": excel_file_name}
    )
    print(f"doc excel: {doc_excel}")
    doc_data.append(doc_excel)

print(f"解析文档个数：{len(doc_data)}")

doc excel: page_content='|序号|地区(城市)|地区(城市)|部级|司局级|其他人员|旺季地区|旺季期间|部级|司局级|其他人员|
|-------:|:-------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------|-------:|---------:|-----------:|:-------------------------------------------------|:--------------|:-------|:---------|:-----------|
|1|北京|全市|1100|650|500||||||
|2|天津|6个中心城区、滨海新区、东丽区、西青区、津南区、北辰区、武清区、宝坻区、静海区、蓟县|800|480|380||||||
|2|天津|宁河区|600|350|320||||||
|3|河北|石家庄市、张家口市、秦皇岛市、廊坊市、承德市、保定市|800|450|350|张家口市|7-9月、11-3月|1200|675|525|
|3|河北|石家庄市、张家口市、秦皇岛市、廊坊市、承德市、保定市|800|450|350|秦皇岛市|7-8月|1200|680|500|
|3|河北|石家庄市、张家口市、秦皇岛市、廊坊市、承德市、保定市|800|450|350|承德市|7-9月|1000|580|580|
|3|河北|其他地区|800|450|310||||||
|4|山西|太原市、大同市、晋城市|800|480|350||||||
|4|山西|临汾市|800|480|330||||||
|4|山西|阳泉市、长治市、晋中市|800|480|310||||||
|4|山西|其他地区|800|400|240||||||
|5|内蒙古|呼和浩特市|800|460|350||||||
|5|内蒙古|其他地区|800|460|320|海拉尔市、满洲里市、阿尔山市|7-9月|1200|690|480|
|5|内蒙古|其他地区|800|460|32

/Users/hedon/mycode/python/regulation_rag/.venv/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [6]:
from model import RagEmbedding
embedding_cls = RagEmbedding()

In [7]:
chroma_client = chromadb.Client()

In [8]:
embedding_db = Chroma.from_documents(doc_data,
                      embedding_cls.get_embedding_fun(),
                      client=chroma_client,
                      collection_name="zhidu_db")

In [9]:
query = "迟到有什么规定？"

In [10]:
related_docs = embedding_db.similarity_search(query=query, k=2)

In [11]:
prompt_template = """
你是企业员工助手，熟悉公司考勤和报销标准等规章制度，需要根据提供的上下文信息 context 来回答员工的提问。\
请直接回答问题，如果上下文信息 context 中没有和问题相关的信息，请直接回答[不知道，请咨询 HR] \
问题：{question}
"{context}"
回答：
"""

In [12]:
from model import RagLLM

llm = RagLLM()

def run_rag_pipeline(queries, k=3):
    for query in queries:
        related_docs = embedding_db.similarity_search(query=query, k=k)
        context = "\n".join([f"上下文{i+1}: {doc.page_content} \n" \
                            for i, doc in enumerate(related_docs)])
        print()
        print()
        llm_prompt = prompt_template.replace("{question}", query).replace("{context}", context)
        print("#"*100)
        print(f"query: {query}")
        print(f"prompt: {llm_prompt}")
        response = llm(llm_prompt)
        print("#"*100)
        print(f"response: {response}")

queries = ["加班有加班费吗？"]
run_rag_pipeline(queries=queries)



####################################################################################################
query: 加班有加班费吗？
prompt: 
你是企业员工助手，熟悉公司考勤和报销标准等规章制度，需要根据提供的上下文信息 context 来回答员工的提问。请直接回答问题，如果上下文信息 context 中没有和问题相关的信息，请直接回答[不知道，请咨询 HR] 问题：加班有加班费吗？
"上下文1: 常上班，以加班计算：
(1)法定节假日
(2)周六、周日
2、正常工作日，延长工作时间连续达到4小时以上的方可计算加班。（加班需提前申请），
加班需有打卡记录，无打卡记录支撑的加班时间，不计加班。加班费按照实际加班时长的2
倍计算。
3、加班以调休等额返还（代替），凡调休人员应填写《请假申请表》，选择“调休”一
栏，经所在部门分管领导签字后，交由人事处核实备案。 

上下文2: 给人事部核算考勤。
9、以下情况视同出勤：持有学校书面通知，经部门负责人同意参加家长会的。
以上，依本规则所请各假如发现有虚假行为者，一经查明，除以旷工论处外，并根据学
校奖惩规定予以处罚。
第四节 加班和调休规定
1、学校以下列日期为例行公休日(若有变更需事先公布)，但因学校工作需要可指定照
常上班，以加班计算：
(1)法定节假日
(2)周六、周日 

上下文3: 栏，经所在部门分管领导签字后，交由人事处核实备案。
4、行政岗及教辅岗都需要参与法定节假日轮流值班。
第五节 违规违纪处理
1. 迟到、早退
(1)劳动考勤是公司支付薪资的依据和职工年度岗位考核内容之一。（2）迟到或早退60
分钟以上（含60分钟），每次视同缺勤1天。（3）职工迟到、早退、脱岗累计超过3次的（含），
从第1次起，每次扣减工资50元。
2. 缺打卡 
"
回答：

####################################################################################################
response: 根据上下文1的信息，加班是有加班费的。具体来说，加班费按照实际加班时长的2倍计算。但是加班需要提前申请，并且需要有打卡记录，无打卡